In [7]:
def weather_api(lat, long):

    # Define the base URL for the API
    base_url = "https://api.open-meteo.com/v1/forecast"

    # Define the parameters in a dictionary
    params = {
        'latitude': f'{lat}',
        'longitude': f'{long}',
        'forecast_days': '16',
        'daily': ','.join([
            'temperature_2m_max',
            'temperature_2m_min',
            'precipitation_sum',
            'rain_sum',
            'showers_sum',
            'snowfall_sum',
            'precipitation_hours',
            'precipitation_probability_max',
            'precipitation_probability_min',
            'precipitation_probability_mean',
            'sunrise',
            'sunset',
            'sunshine_duration',
            'daylight_duration',
            'wind_speed_10m_max',
            'wind_gusts_10m_max',
            'wind_direction_10m_dominant',
            'shortwave_radiation_sum',
            'et0_fao_evapotranspiration',
            'uv_index_max',
            'uv_index_clear_sky_max',
        ])
    }

    # Make the GET request with the base URL and the parameters
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        # Process the response data
        return data
    else:
        raise Exception(f"Request failed with status code: {response.status_code}")

In [8]:
def format_weather_data(weather_data):

    # Extract daily weather data
    daily_data = weather_data.get('daily', {})
    time_data = daily_data.get('time', [])

    # Prepare a list to hold the formatted strings for each time entry
    formatted_entries = []

    # Loop through each time entry and collect the corresponding weather features
    for index, time_entry in enumerate(time_data):
        features = [f"Time: {time_entry}"]
        for feature, values in daily_data.items():
            if feature != 'time':  # Skip the 'time' key since we already have it
                value = values[index]
                features.append(f"{feature}: {value}")
        formatted_entries.append('\n'.join(features))

    # Join all entries separated by two newlines (for readability)
    formatted_text = '\n\n'.join(formatted_entries)

    # Print the formatted text
    return formatted_text

In [9]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document

def get_text_chunks_langchain(text):
   text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
   docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
   return docs



In [10]:
import requests
from urllib.parse import quote_plus

def get_lat_lon(location_name):
    # Encode the location name to be URL-friendly
    encoded_location_name = quote_plus(location_name)

    # Use the OpenStreetMap Nominatim API for geocoding
    url = f"https://nominatim.openstreetmap.org/search?format=json&q={encoded_location_name}"

    # Make an HTTP request to the service
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Check if any results were returned
        if data:
            # Extract the latitude and longitude from the first result
            latitude = float(data[0]['lat'])
            longitude = float(data[0]['lon'])
            return latitude, longitude
        else:
            raise Exception(f"No results found for location '{location_name}'")
    else:
        raise Exception("Failed to connect to the geocoding service")

# Usage example:
try:
    location = "Delhi"
    lat, lon = get_lat_lon(location)
    print(f"The latitude and longitude of {location} are {lat}, {lon}")
except Exception as e:
    print(e)

The latitude and longitude of Delhi are 28.6273928, 77.1716954


In [2]:
from threading import Thread
from typing import Optional

import gradio as gr
from langchain import PromptTemplate, LLMChain
from langchain.llms.base import LLM
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline
)


In [5]:
import torch

In [3]:
def load_llm():

    #Loading the Mistral Model
    model_name='mistralai/Mistral-7B-Instruct-v0.2'
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )


    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        )

    # Building a LLM text-generation pipeline
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.2,
        repetition_penalty=1.1,
        return_full_text=True,
        max_new_tokens=1024,
    )


    return text_generation_pipeline


In [6]:
llm = load_llm()

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
from qdrant_client import QdrantClient
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant

def add_to_vectorstore(location):

    lat, long = get_lat_lon(location)
    data = weather_api(lat, long)
    formatted_data = format_weather_data(data)
    docs = get_text_chunks_langchain(formatted_data)

    client = QdrantClient("localhost", port=6333)
    client.delete_collection(collection_name="my_weather_data")

    global qdrant
    qdrant = Qdrant.from_documents(
        docs,
        HuggingFaceEmbeddings(),
        url="http://localhost:6333",
        prefer_grpc=True,
        collection_name="my_weather_data",
    )

    return f"Weather data for latitude: {lat} & longitude: {long} added"


In [15]:
def answer_query(message, chat_history):
    context_docs = qdrant.similarity_search(message, k= 16)
    context = ' '.join(doc.page_content for doc in context_docs)

    template = f"""Answer the question based only on the following context:
        {context}


        Question: {message}
    """


    result = llm(template)


    answer = result[0]["generated_text"].replace(template, '')


    chat_history.append((message, answer))


    return "", chat_history

Created a chunk of size 563, which is longer than the specified 500
Created a chunk of size 563, which is longer than the specified 500
Created a chunk of size 564, which is longer than the specified 500
Created a chunk of size 565, which is longer than the specified 500
Created a chunk of size 560, which is longer than the specified 500
Created a chunk of size 564, which is longer than the specified 500
Created a chunk of size 562, which is longer than the specified 500
Created a chunk of size 564, which is longer than the specified 500
Created a chunk of size 565, which is longer than the specified 500
Created a chunk of size 561, which is longer than the specified 500
Created a chunk of size 572, which is longer than the specified 500
Created a chunk of size 574, which is longer than the specified 500
Created a chunk of size 574, which is longer than the specified 500
Created a chunk of size 570, which is longer than the specified 500
Created a chunk of size 571, which is longer tha

In [16]:
with gr.Blocks() as demo:

    loc = gr.Textbox(label= "Enter the location for weather data, for e.g Eiffel Tower, Delhi")
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    loc.submit(add_to_vectorstore, loc, loc)
    msg.submit(answer_query, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)



In [17]:
demo.queue()
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://a0f3ac923c4cf1ee4b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Created a chunk of size 563, which is longer than the specified 500
Created a chunk of size 563, which is longer than the specified 500
Created a chunk of size 564, which is longer than the specified 500
Created a chunk of size 565, which is longer than the specified 500
Created a chunk of size 560, which is longer than the specified 500
Created a chunk of size 564, which is longer than the specified 500
Created a chunk of size 562, which is longer than the specified 500
Created a chunk of size 564, which is longer than the specified 500
Created a chunk of size 565, which is longer than the specified 500
Created a chunk of size 561, which is longer than the specified 500
Created a chunk of size 572, which is longer than the specified 500
Created a chunk of size 574, which is longer than the specified 500
Created a chunk of size 574, which is longer than the specified 500
Created a chunk of size 570, which is longer than the specified 500
Created a chunk of size 571, which is longer tha